In [ ]:
!pkill -f streamlit #Find and stop existing Streamlit processes to avoid conflic
ngrok.kill()
# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(public_url)

# Run Streamlit app
#!streamlit run chatbot.py --server.port 8501 > /content/logs.txt 2>&1 &
!python -u -m streamlit run chatbot.py --server.port 8501 > /content/logs.txt 2>&1 &
# ngrok.get_tunnels()

NgrokTunnel: "https://4402-34-46-229-42.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
#!ln -s / root #Create a link to the root directory in Google Colab
from google.colab import files
uploaded = files.upload() #Choose secrets.toml
!mkdir -p /root/.streamlit/
!mv secrets.toml /root/.streamlit/
# Create or update a config.toml file in the .streamlit directory with the logging level set to debug
#!echo -e "[logger]\nlevel = 'debug'" > /root/.streamlit/config.toml
#!pip install -r requirements.txt
!pip install streamlit pyngrok pandas openai pinecone-client python-dotenv supabase > /dev/null 2>&1
from pyngrok import ngrok
!ngrok config set --no-auth > /dev/null 2>&1
!ngrok authtoken 2lu4WybNd7R2foCiwPxyKkrP8jw_5Nhf922w71PZQKxMRqj3Z > /dev/null 2>&1

Saving secrets.toml to secrets.toml


In [ ]:
import os
if os.path.exists('/root/.streamlit/secrets.toml'):
  print("OK")
else: print("Not OK")

OK


In [ ]:
#!pip install streamlit-folium folium geopy > /dev/null 2>&1
#!pip install streamlit streamlit-folium geopy requests > /dev/null 2>&1
!pip install streamlit streamlit-folium requests geogy > /dev/null 2>&1

In [ ]:
from geopy.geocoders import Nominatim
def get_lat_lon(address):
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(address)
    return location.latitude, location.longitude

# Example address
address = "1600 Amphitheatre Parkway, Mountain View, CA"
lat, lon = get_lat_lon(address)


In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_folium import st_folium
import folium
# import requests
from geopy.geocoders import Nominatim

# Function to get address from latitude and longitude using OpenCage Geocoding API
"""def get_address(lat, lon, api_key):
    url = f"https://api.opencagedata.com/geocode/v1/json?q={lat}+{lon}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        response_json = response.json()
        if response_json['results']:
            return response_json['results'][0]['formatted']
        else:
            return "Address not found"
    else:
        return f"Geocoding service error: {response.status_code}"
"""
def get_address(lat, lon):
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.reverse((lat, lon), exactly_one=True)
    return location.address if location else "Address not found"

# Input for OpenCage API key
api_key = "0f6e1db8439e4c93b119873a7c61804b"

# Function to get latitude and longitude from address
def get_lat_lon(address):
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(address)
    return location.latitude, location.longitude

# Example address
address = "1600 Amphitheatre Parkway, Mountain View, CA"
lat, lon = get_lat_lon(address)

# Initialize the map only once
if 'map' not in st.session_state:
    st.session_state['map'] = folium.Map(location=[lat, lon], zoom_start=10)
    st.session_state['last_clicked'] = None  # To track the last click

# Display the map and capture click events
st_data = st_folium(st.session_state['map'], width=700, height=500, key="map_display")

# Handle click events
if st_data and st_data.get('last_clicked'):
    lat = st_data['last_clicked']['lat']
    lon = st_data['last_clicked']['lng']

    if st.session_state['last_clicked'] != (lat, lon):
        st.session_state['last_clicked'] = (lat, lon)
        address = get_address(lat, lon, api_key)

        st.write(f"Latitude: {lat}, Longitude: {lon}")
        st.write(f"Address: {address}" if address else "Address not found")

        # Update the map with the latest marker
        st.session_state['map'] = folium.Map(location=[lat, lon], zoom_start=10)
        folium.Marker(location=[lat, lon], popup=address).add_to(st.session_state['map'])

        # Render the updated map again
        st_data = st_folium(st.session_state['map'], width=700, height=500, key="map_display")


Overwriting app.py


In [ ]:
%%writefile app.py
import streamlit as st
from streamlit_folium import st_folium
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderServiceError

# Function to get address from latitude and longitude
def get_address(lat, lon):
  try:
    geolocator = Nominatim(user_agent="geoapiExercises")
    location = geolocator.reverse((lat, lon), language='en')
    return location.address if location else None
  except GeocoderServiceError:
        return None
# Create a map centered around a specific location
m = folium.Map(location=[37.7749, -122.4194], zoom_start=10)
m.add_child(folium.LatLngPopup())
# Add a click event listener to get latitude and longitude
def on_click(event):
    folium.Marker(location=event.latlng).add_to(m)
    st.session_state['latitude'], st.session_state['longitude'] = event.latlng
    st.write(f"Clicked at: {event.latlng}")

# Display the map
st_data = st_folium(m, width=700, height=500)

if st_data is not None and 'last_clicked' in st_data and st_data['last_clicked'] is not None:
    lat = st_data['last_clicked']['lat']
    lon = st_data['last_clicked']['lng']
    # Add a marker with a custom icon (water drop symbol)
    icon = folium.Icon(icon='tint', prefix='fa', color='red')
    folium.Marker(location=[lat, lon], icon=icon).add_to(m)
    address = get_address(lat, lon)
    st.write(f"Latitude: {lat}, Longitude: {lon}")
    if address:
        st.write(f"Address: {address}")
    else:
        st.write("Address not found")
else:
    st.write("Click on the map to get the address.")

Overwriting app.py


In [ ]:
!pkill -f streamlit #Find and stop existing Streamlit processes to avoid conflic
ngrok.kill()
# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(public_url)
!python -u -m streamlit run app.py --server.port 8501 > /content/logs.txt 2>&1 &

NgrokTunnel: "https://9a26-34-147-101-165.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
%%writefile chatbot.py
import sys
import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
# Set the watchdog logger to suppress debug messages
watchdog_logger = logging.getLogger("watchdog")
watchdog_logger.setLevel(logging.ERROR)
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("numexpr").setLevel(logging.WARNING)
logging.getLogger("pinecone_plugin_interface").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("hpack").setLevel(logging.WARNING)
logging.getLogger("pinecone.core.openapi.shared.rest").setLevel(logging.WARNING)
print("Starting Streamlit App") # file=sys.stdout

import streamlit as st
import pandas as pd
import random
import time
from openai import OpenAI
#import textwrap3 as textwrap
import dotenv
from dotenv import load_dotenv
import numpy as np
import os
from pinecone import Pinecone
import csv
import json
from supabase import create_client
from datetime import datetime, timezone
from geopy.geocoders import Nominatim

# For sending email
# import sendgrid
# from sendgrid import SendGridAPIClient
# from sendgrid.helpers.mail import Mail
# from joblib import load
# For getting google sheet content
# import requests
# from bs4 import BeautifulSoup
# For update knowledge base
# import Update_KB

embedding_model = "text-embedding-ada-002"

try:
    OPENAI_API_KEY = st.secrets["OPENAI_API_KEY"]
    PINECONE_API_KEY = st.secrets["PINECONE_API_KEY"]
    SENDGRID_API_KEY = st.secrets["SENDGRID_API_KEY"]
    SUPABASE_KEY = st.secrets["SUPABASE_KEY"]
except Exception as e:
    # Secrets not found in Streamlit, try loading from local .env file
    load_dotenv()
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
    PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
    SENDGRID_API_KEY = os.getenv("SENDGRID_API_KEY")
    SUPABASE_KEY = os.getenv("SUPABASE_KEY")
    if not OPENAI_API_KEY or not PINECONE_API_KEY or not SENDGRID_API_KEY or not SUPABASE_KEY:
        st.error("Environment file error or secrets not found!")
        st.error(e)
# Set OpenAI API key
client = OpenAI(api_key = OPENAI_API_KEY)

index_name = 'cstugpt-kb'
pc = Pinecone( # initialize connection to pinecone
    api_key=PINECONE_API_KEY,
    environment="us-west1-gcp-free")
try:
    pincone_index = pc.Index(index_name) # connect to pinecone index
except:
    print("Could not connect to Pinecone")
    pincone_index = None

supabase = create_client("https://ziaisnybjhkjcbprxava.supabase.co", SUPABASE_KEY)
product_category_list = ["All"]
#def fetch_categories():
# Fetches category names from the public.categories table and stores them in the global categories_list."""
  # global categories_list
try:
        # Query the categories table to fetch category names
        response = supabase.table("categories").select("name").execute()
        if response.data:
            # Extract category names from the query result
            product_category_list += [category['name'] for category in response.data]
        else:
            product_category_list = ["No categories found"]
except Exception as e:
        st.error(f"Error fetching categories: {e}")

# Call the function to fetch categories when the app starts
# fetch_categories()

# Function to update filters
def update_filters(function_args):
    """Parses user input and creates filters."""
    #allow_inspect_value = function_args.get("allow_inspect")
    #allow_inspect_value = "Yes" if allow_inspect_value is True else "No" if allow_inspect_value is False else None
    st.session_state.deal_type = function_args.get("deal_type", "All")
    st.session_state.product_name = function_args.get("product_name", "")
    st.session_state.location = function_args.get("location", "")
    st.session_state.search_radius = function_args.get("search_radius", 0.0)
    st.session_state.num_days = str(function_args.get("num_days", "")) #Text field
    st.session_state.condition = function_args.get("condition", "All")
    st.session_state.status = function_args.get("status", "All")
    st.session_state.allow_inspect = function_args.get("allow_inspect", "All")
    st.session_state.delivery_method = function_args.get("delivery_method", "All")
    st.session_state.min_price = function_args.get("min_price", 0.0)
    st.session_state.max_price = str(function_args.get("max_price", ""))  #Text field 2000000000.0
    st.session_state.category_name = function_args.get("category_name", "All")
    st.session_state.product_description = function_args.get("product_description", "")

    filters = {
        "_category_name": function_args.get("category_name", None),
        "_product_description": function_args.get("product_description", None),
        "_deal_type": function_args.get("deal_type", None),
        "_min_price": function_args.get("min_price", None),
        "_max_price": function_args.get("max_price", None),
        "_condition": function_args.get("condition", None),
        "_status": function_args.get("status", None),
        "_product_name": function_args.get("product_name", None),
        "_allow_inspect": function_args.get("allow_inspect", None),
        "_delivery_method": function_args.get("delivery_method", None),
        "_address": function_args.get("address", None),
        "_search_radius": function_args.get("search_radius", None),
        "_num_days": function_args.get("num_days", None),
    }
    return filters

# Function to fetch products
def fetch_product(filters):
    columns_to_display = ["name", "description", "deal_type", "price", "published_at", "condition",
                          "status", "rating", "allow_inspect", "delivery_method", "address", "quantity", "product_image_urls"]
    try:
        product_data, count = supabase.rpc("filter_products", filters).execute()
        if product_data[1]:
            products_list = []
            # Construct the product display table
            for item in product_data[1]:
                if item.get("type") == "Buyer":
                    min_budget = item.get("min_budget")
                    max_budget = item.get("max_budget")
                    item["price"] = f"${min_budget} - ${max_budget}" if min_budget and max_budget else ""
                elif item.get("type") == "Seller":
                    price = item.get("price")
                    item["price"] = f"${price}" if price is not None else ""

                # Format dates and links
                if "published_at" in item and item["published_at"]:
                    try:
                        published_at_date = datetime.fromisoformat(item["published_at"])
                        now_utc = datetime.now(timezone.utc)
                        days_since_posting = (now_utc - published_at_date).days
                        item["published_at"] = f"{days_since_posting} days ago"
                    except ValueError:
                        item["published_at"] = "Invalid date"

                # Create a link to Google Maps for the address
                latitude = item.get("latitude")
                longitude = item.get("longitude")
                address = item.get("address")
                if latitude and longitude:
                    map_url = f"https://google.com/maps?q={latitude},{longitude}"
                    #item["address"] = f"<a href='{map_url}' target='_blank'>{address}</a>"

                # Add item to the product list
                products_list.append({
                    "PRODUTCT NAME": item.get('name', ''),
                    "DESCRIPTION": item.get('description', ''),
                    "DATA PROVIDER": item.get('deal_type', ''),
                    "PRICE": item.get('price', ''),
                    "CONDITION": item.get('condition', ''),
                    "STATUS": item.get('status', ''),
                    "LISTING DURATION": item.get('published_at', ''),
                    "INSPECTABLE": item.get('allow_inspect', ''),
                    "DELIVERY METHOD": item.get('delivery_method', ''),
                    "ADDRESS": item.get('address', ''),
                    "MAP LINK": map_url
                })
            df = pd.DataFrame(products_list)
            # Add a sequence number column starting from 1
            df.index = df.index + 1
            df.index.name = 'No.'
            return df
        else:
            return "No products match your query."
    except (ValueError, TypeError, Exception) as e:
        return f"Error: {str(e)}"

def coordinate2location(lat, lon):
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.reverse((lat, lon), exactly_one=True)
    return location.address if location else "Location not found"

def location2coordinate(addr):
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(addr)
    return location.latitude, location.longitude

# Initialize history and state for chatbot
MAX_OUTPUT_TOKEN = 300
MAX_PROMPT_HISTORY_LENGTH = 30
# Initialize default values in session state (this only runs the first time)
def initialize():
  if 'initialized' in st.session_state:
    if len(st.session_state.prompt_history) > MAX_PROMPT_HISTORY_LENGTH:
      st.session_state.prompt_history = st.session_state.prompt_history[-MAX_PROMPT_HISTORY_LENGTH:]
  else:
    st.session_state.initialized = True
    st.session_state.chat_history = []
    st.session_state.prompt_history = [{"role": "system", "content": ("You are Matchder marketing assistant. "
      "If users search, buy or sell products, suggest they clarify any unclear inputs and ask if they can provide more info or that's all. If that's all, confirm final query with them before calling search_product(). "
      "Note that deal_type = 'Seller' if user is a buyer or finds sale items, deal_type = 'Buyer' if user is a seller or finds purchase items. "
      "If you have not enough info to answer their query, refer them to Matchder website dev.matchder.com")}]
    st.session_state.deal_type = "All"
    st.session_state.product_name = ""
    st.session_state.address = ""
    st.session_state.num_days = 365
    st.session_state.condition = "All"
    st.session_state.status = "All"
    st.session_state.allow_inspect = "All"
    st.session_state.delivery_method = "All"
    st.session_state.min_price = 0.0
    st.session_state.max_price = 2000000000.0
    st.session_state.category_name = "All"
    st.session_state.product_description = ""
initialize()
# STREAMLIT INTERFACE
with st.sidebar:
  #st.sidebar.image("CSTU.png", use_column_width=True)
  #st.sidebar.image("robo.gif", use_column_width=True)
  # Set the text color and alignment for the selectbox label and options
  st.markdown("""
    <style>
    .stSelectbox > label,
    .stSelectbox > select {
      color: darkblue;
      font-weight: bold;
    }
    </style>
    """, unsafe_allow_html=True)
  st.markdown("<font color='darkblue'><b><p style='font-size: 21px; text-align: center; line-height: 0;'>MATCHDER CHATBOT</p></b></font>", unsafe_allow_html=True)
  st.write("<font color='darkblue'><b><p style='text-align: center; line-height: 0;'>NEW CHAT SESSION</p></b></font>", unsafe_allow_html=True)
  col1, col2, col3 = st.columns([1, 1, 1])
  with col2:
    if st.button(" 🗑️ "):
      initialize()
      st.rerun()
  st.markdown("<font color='darkblue'><b><p style='text-align: center; line-height: 0;'>KNOWLEDGE-BASE SETUP</p></b></font>", unsafe_allow_html=True)
  n_KB = int(st.selectbox("Number of KB records to find/query:", ["1", "2", "3", "4", "5"]))
  # Create a menu
  if 'index' not in st.session_state:
      st.session_state['index'] = None
  st.markdown("<font color='darkblue'><b><p style='text-align: center; line-height: 0;'>DATABASE MANAGEMENT</p></b></font>", unsafe_allow_html=True)
  options = st.selectbox(
      'Choose a task option:',
      ('UPDATE KNOWLEDGE-BASE', '', ''),
      key='menu',
      index = st.session_state['index']
  )
  if options is not None:
      st.session_state['index'] = ('UPDATE KNOWLEDGE-BASE', '', '').index(options)
      #expander=st.sidebar
      if 'login_status' not in st.session_state:
          st.session_state['login_status'] = False
      st.session_state['key'] = 0
      with st.expander:
          username = st.text_input("Enter your username:", key=st.session_state['key'])
          password = st.text_input("Enter your password:", type="password", key=st.session_state['key']+1)
          #upload_window = st.empty()
          if st.button("LOGIN"):
              for user in st.secrets["admin_accounts"].get("users", []):
                  if username == user.get("username") and password == user.get("password"):
                      st.session_state['login_status'] = True
                      st.balloons()
                      break
              if st.session_state['login_status'] == False:
                  st.warning("Invalid username or password. Please try again!")
          if st.button("LOGOUT"):
              st.session_state['login_status'] = False
              st.session_state['key'] += 2
              st.session_state['index'] = None
              st.rerun()
          if st.session_state['login_status']:
              if options == 'UPDATE KNOWLEDGE-BASE':
                file_name = st.file_uploader("Select your pdf catalog file", type="pdf")
                if file_name is not None:
                    st_button=st.empty()
                    if st_button.button("UPLOAD"):
                        st_button.empty()
                        try:
                              result=update_kb_openai()
                              st.balloons()
                              st.success("Knowledge-base records updated:\n"+result)
                        except Exception as e:
                              st.write(e)

# Main interface
st.markdown("""
    <style>
    .stChatInput > div {
        border: 1px solid black !important;
        border-radius: 10px !important;
        resize: both !important;
        overflow: auto !important;
    }
    .stButton > button {
        width: 45px;
        height: 45px;
        font-size: 25px;}
    </style>
""", unsafe_allow_html=True)

#with st.container():
col1, col2 = st.columns([50, 1])
with col1:
    with st.expander("SEARCH FOR PRODUCT BASED ON USER-SELECTED FILTERS"):
      st.session_state.deal_type = st.radio("Are you looking for", ["Buyer", "Seller", "All"],
         index=["Buyer", "Seller", "All"].index(st.session_state.deal_type) if st.session_state.deal_type in ["Buyer", "Seller", "All"] else 2)
      st.session_state.product_name = st.text_input("Product Name", value=st.session_state.product_name)
      st.session_state.address = st.text_input("Address", value=st.session_state.address)
      st.session_state.search_radius = st.number_input("Address", min_value=0.0, value=st.session_state.address)
      st.session_state.num_days = st.text_input("Within number of days since advertised", value=st.session_state.num_days)
      st.session_state.condition = st.radio("Condition", ["All", "New", "Like New", "Excellent", "Good", "Fair", "Salvage"],
         index=["All", "New", "Like New", "Excellent", "Good", "Fair", "Salvage"].index(st.session_state.condition) if st.session_state.condition in ["All", "New", "Like New", "Excellent", "Good", "Fair", "Salvage"] else 0)
      st.session_state.status = st.radio("Status", ["All", "Available", "Sold out", "Still searching", "Bought"],
         index=["All", "Available", "Sold out", "Still searching", "Bought"].index(st.session_state.status) if st.session_state.status in ["All", "Available", "Sold out", "Still searching", "Bought"] else 0)
      allow_inspect_options = {"All": None, "Yes": True, "No": False}
      allow_inspect_selection = st.radio("Pre-purchase Inspection Allowed", ["All", "Yes", "No"],
         index=["All", "Yes", "No"].index("Yes" if st.session_state.allow_inspect is True else "No" if st.session_state.allow_inspect is False else "All"))
      st.session_state.allow_inspect = allow_inspect_options[allow_inspect_selection]  # Store the boolean value
      st.session_state.delivery_method = st.radio("Delivery Method", ["All", "Pickup", "Delivery Driver", "Shipping"],
         index=["All", "Pickup", "Delivery Driver", "Shipping"].index(st.session_state.delivery_method) if st.session_state.delivery_method in ["All", "Pickup", "Delivery Driver", "Shipping"] else 0)
      st.session_state.min_price = st.number_input("Min Price", min_value=0.0, value=float(st.session_state.min_price), format="%.2f")
      st.session_state.max_price = st.text_input("Max Price", value=st.session_state.max_price)
      st.session_state.category_name = st.selectbox("Category Name", product_category_list,
         index=product_category_list.index(st.session_state.category_name) if st.session_state.category_name in product_category_list else 0)
      st.session_state.product_description = st.text_input("Product Description", value=st.session_state.product_description)
search_result = None
with col2:
    #st.markdown("<br><br>", unsafe_allow_html=True)
    #submit_button = st.markdown('<button title="Submit your question"> ⏎ </button>', unsafe_allow_html=True)
    #submit_button = st.button(" ⏎ ", key="submit_button")
    #if st.markdown('<button title="Search products">🔍</button>', unsafe_allow_html=True): #Search button
    if st.button("🔍", key="search_button"):
          def reverse_map(value):
              """Map 'All' and '' to None, 'Yes' to True, 'No' to False."""
              if value in ["All", "", 0]:
                  return None
              elif value == "Yes":
                  return True
              elif value == "No":
                  return False
              return value
          def str_to_int(str): try: return int(str) except: return None
          def str_to_float(str): try: return float(str) except: return None
          filters = {
              "_category_name": reverse_map(st.session_state.category_name),
              "_product_description": reverse_map(st.session_state.product_description),
              "_deal_type": reverse_map(st.session_state.deal_type),
              "_min_price": reverse_map(st.session_state.min_price),
              "_max_price": str_to_float(st.session_state.max_price),
              "_condition": reverse_map(st.session_state.condition),
              "_status": reverse_map(st.session_state.status),
              "_product_name": reverse_map(st.session_state.product_name),
              "_allow_inspect": reverse_map(st.session_state.allow_inspect),
              "_delivery_method": reverse_map(st.session_state.delivery_method),
              "_address": reverse_map(st.session_state.address),
              "_search_radius": reverse_map(st.session_state.search_radius),
              "_num_days": str_to_int(st.session_state.num_days),
          }
          search_result = fetch_product(filters)
if isinstance(search_result, pd.DataFrame):  # If it's a DataFrame, display it as a table
    st.write("Products match your selected filters are listed as follows:\n")
    st.dataframe(search_result, column_config={'MAP LINK': st.column_config.LinkColumn()})
    st.write("________________________________________________________________________________________________________________")
    #st.markdown(search_result.to_html(escape=False, index=False), unsafe_allow_html=True)
elif isinstance(search_result, str):  # If it's a text message, display the text
    st.write(search_result)

#st.markdown('<div class="chat-container">', unsafe_allow_html=True)
if st.session_state.chat_history:
    prev_role = None  # Track the previous chat role
    for chat in st.session_state.chat_history:
        with st.chat_message(chat["role"]):
            if chat["role"] == "user" and prev_role != "user": st.markdown("**YOU:**")
            elif chat["role"] == "assistant" and prev_role != "assistant": st.markdown("**MATCHDER:**")
            elif chat["role"] == "system" and prev_role != "system": st.markdown("**SYSTEM:**")
            if isinstance(chat["content"], list):
                st.write("Products match your query are listed as follows:\n")
                st.dataframe(pd.DataFrame(chat["content"]), column_config={'MAP LINK': st.column_config.LinkColumn()})
            else: st.write(chat["content"])
            prev_role = chat["role"]  # Update the previous role
#st.markdown('</div>', unsafe_allow_html=True)

#user_input = st.text_area("ENTER YOUR QUESTION", placeholder="Type here ...", key='user_input', height=1, help='Resize by dragging the bottom-right corner', max_chars=None,)
user_input = st.chat_input("Type your question here ...")

# CHAT PROCESS
# Chatbot function
def chat(user_input):
    st.session_state.prompt_history.insert(-1, {"role": "user", "content": user_input})
    st.session_state.chat_history.append({"role": "user", "content": user_input})
    try:
        response1 = client.chat.completions.create(
            model="gpt-4o",
            messages=[msg for msg in st.session_state.prompt_history],
            max_tokens = MAX_OUTPUT_TOKEN,
            functions=[{
                "name": "search_product",
                "description": "Search products from the database based on user input",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "category_name": {"type": "string", "description": f"Product category chosen from: {', '.join(product_category_list)}"},
                        "product_name": {"type": "string", "description": "Product name"},
                        "product_description": {"type": "string", "description": "Product description"},
                        "deal_type": {"type": "string", "description": "deal_type = 'Seller' if user is a buyer or finds sale items, deal_type = 'Buyer' if user is a seller or finds purchase items"},
                        "min_price": {"type": "number", "description": "Product min price in US dollars"},
                        "max_price": {"type": "number", "description": "Product max price in US dollars"},
                        "condition": {"type": "string", "description": "Product condition: New, Like New, Excellent, Good, Fair, Salvage"},
                        "status": {"type": "string", "description": "Product status: Available, Sold out, Still searching, Bought"},
                        "allow_inspect": {"type": "boolean", "description": "Whether allow pre-purchase inspection"},
                        "delivery_method": {"type": "string", "description": "Delivery method: Pickup, Delivery Driver, Shipping"},
                        "address": {"type": "string", "description": "Address"},
                        "search_radius:": {"type": "number", "description": "Maximum distance in miles to search within the map area from the address"}
                        "num_days": {"type": "integer", "description": "The maximum number of days since the product was posted, indicating that the user wants to search for products posted within this number of days from today"},
                    }
                }
            }]
        )
        #"deal_type": {"type": "string", "description": "deal_type is 'Seller' if user asks to buy/find items posted for sale, or 'Buyer' if user asks to sell/find items posted for purchase"}
        print(response1.choices[0].message, file=sys.stdout)
        content1 = response1.choices[0].message.content
        if content1: # Keep the system message at last of prompt history
            st.session_state.prompt_history.insert(-1, {"role": "assistant", "content": content1})
            st.session_state.chat_history.append({"role": "assistant", "content": content1})

        if response1.choices[0].finish_reason == "function_call":
            function_name = response1.choices[0].message.function_call.name
            function_args = json.loads(response1.choices[0].message.function_call.arguments)

            if function_name == "search_product":
                filters = update_filters(function_args)
                search_result = fetch_product(filters)
                if isinstance(search_result, pd.DataFrame): search_result = search_result.to_dict(orient='records')  # Convert DataFrame to a list of JSON-friendly dict
                #else: search_result = [{"Result": search_result}]
                print(function_args, file=sys.stdout)
                response2 = client.chat.completions.create(# Send the search result back to LLM to allow the model to reply to the user
                    model="gpt-4o",
                    messages=[{"role": "system", "content": "Summarize following results, beginning with: 'Here is the result based on your request:'"},
                              {"role": "function", "name": "search_product", "content": response1.choices[0].message.function_call.arguments},
                              {"role": "function", "name": "search_product", "content": json.dumps(search_result)}],
                    max_tokens = MAX_OUTPUT_TOKEN)
                print(response2.choices[0].message, file=sys.stdout)
                st.session_state.chat_history.append({"role": "assistant", "content": search_result}) # Table in form of list of dictionary
                st.session_state.chat_history.append({"role": "assistant", "content": response2.choices[0].message.content})
                st.session_state.prompt_history.insert(-1,{"role": "assistant", "content": response2.choices[0].message.content})
                # st.session_state.prompt_history.append({"role": "assistant", "content": search_result})
            else:
                st.write(("system", f"Unhandled function call: {function_name}"))
    except Exception as e:
        st.session_state.chat_history.append({"role": "system", "content": f"An error occurred: {str(e)}"})

# Handle chat submission
if user_input:
  if user_input.strip() != "":
    chat(user_input)
    user_input = ""
    st.rerun()


Writing chatbot.py


In [ ]:
#!streamlit run chatbot.py --server.address=localhost &>/content/logs.txt &
!streamlit run chatbot.py --server.port=8501 &>/content/logs.txt &
!npx localtunnel --port 8501 --subdomain lamdao & curl ipv4.icanhazip.com

34.141.186.66
your url is: https://lamdao.loca.lt


In [ ]:
!ps aux | grep '[s]treamlit' #Checking running instance to kill it

In [ ]:
!pkill -f streamlit #Stop all running instance

In [ ]:
!kill <process_id>

In [ ]:
# Start ngrok tunnel
public_url = ngrok.connect(8501, "http").public_url
print(f"Public URL: {public_url}")

# Add logic to send a request with the ngrok-skip-browser-warning header
headers = {
    "ngrok-skip-browser-warning": "true"
}

# Make a GET request to the public URL with the custom header to bypass the warning
response = requests.get(public_url, headers=headers)

# Run the Streamlit app
!streamlit run chatbot.py &> /dev/null &

Public URL: https://e1e6-34-173-92-239.ngrok-free.app


In [ ]:
delimiter = ""
"""
# Accept user input
if user_input := st.chat_input("Enter a prompt here to ask me for information"):
    # Add user message to chat history
    st.session_state.chat_history.append({"role": "user", "content": user_input})
    if OPENAI_API_KEY:
        # Word2Vector embedding
        # input_emb=generate_embedding(embedding_model, user_input)
        # OpenAI embedding
        res = openai.Embedding.create(input=[user_input],engine=embedding_model)
        input_emb=res['data'][0]['embedding']
        kb_res = pincone_index.query(vector=input_emb, top_k=n_KB, include_metadata=True, namespace='cstu', metric="cosine")
        #If the include_metadata parameter is set to True, the query method will only return the id, score, and metadata for each document. The vector for each document will not be returned
        metadata_text_list = [x['metadata']['text'] for x in kb_res['matches']]
        limit = 3600  #set the limit of knowledge base words
        kb_content = " "
        count = 0
        proceed = True
        while proceed and count < len(metadata_text_list):  # append until hitting limit
            if len(kb_content) + len(metadata_text_list[count]) >= limit:
                proceed = False
            else:
                    kb_content += metadata_text_list[count]
            count += 1

        # Add knowledge base and user message to promt history
        st.session_state.prompt_history.append({"role": "system", "content": f"{delimiter}{kb_content}{delimiter}"})
        st.session_state.prompt_history.append({"role": "user", "content": user_input})

        # Get the model response
        response = chat_complete_messages(st.session_state.prompt_history, temperature=0)

# Display chat messages
for message in st.session_state.chat_history:
        with st.chat_message(message["role"]):
            if isinstance(message["content"], list):
                st.write("Here is your results:")
                df = pd.DataFrame(message["content"])
                st.dataframe(df)
            else:
                st.markdown(message["content"])
"""

'\n# Accept user input\nif user_input := st.chat_input("Enter a prompt here to ask me for information"):\n    # Add user message to chat history\n    st.session_state.chat_history.append({"role": "user", "content": user_input})\n    if OPENAI_API_KEY:\n        # Word2Vector embedding\n        # input_emb=generate_embedding(embedding_model, user_input)\n        # OpenAI embedding\n        res = openai.Embedding.create(input=[user_input],engine=embedding_model)\n        input_emb=res[\'data\'][0][\'embedding\']\n        kb_res = pincone_index.query(vector=input_emb, top_k=n_KB, include_metadata=True, namespace=\'cstu\', metric="cosine")\n        #If the include_metadata parameter is set to True, the query method will only return the id, score, and metadata for each document. The vector for each document will not be returned\n        metadata_text_list = [x[\'metadata\'][\'text\'] for x in kb_res[\'matches\']]\n        limit = 3600  #set the limit of knowledge base words\n        kb_conte

In [ ]:
# Display chat history
"""if "chat_history" in st.session_state:
    for chat in st.session_state.chat_history:
        with st.chat_message("user" if chat[0] == "user" else "assistant"):
            st.write(chat[1])
for chat in st.session_state.chat_history:
    if chat[0] == "user":
        st.write(f"**You**: {chat[1]}")
    elif chat[0] == "assistant":
        st.write(f"**Bot**: {chat[1]}")
"""